**Import** **Libraries**

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import RobustScaler
import umap
from sklearn.manifold import trustworthiness
import hdbscan
from sklearn.metrics import silhouette_score
from sklearn.cluster import SpectralClustering
from scipy import stats
from sklearn.cluster import AffinityPropagation
from sklearn.metrics import davies_bouldin_score

**Read Data**

In [ ]:
# Get the current working directory
current_wd = os.getcwd()

# Define paths relative to the base directory
climate_data_dir = os.path.join(current_wd, "Project")
climate_data = os.path.join(climate_data_dir,"Climate","ClimateDataBasel.csv")

In [ ]:
#Store Features names in feature variable
features = ["Temperature (Min) oC","Temperature (Max) oC.","Temperature (Mean) oC.","Relative Humidity (Min) %","Relative Humidity (Max) %","Relative Humidity (Mean) %","Sea Level Pressure (Min) hPa","Sea Level Pressure (Max) hPa","Sea Level Pressure (Mean) hPa","Precipitation Total mm","Snowfall Amount cm","Sunshine Duration min","Wind Gust (Min) Km/h","Wind Gust (Max) Km/h","Wind Gust (Mean) Km/h","Wind Speed (Min) Km/h","Wind Speed (Max) Km/h","Wind Speed (Mean) Km/h"]
#read csv and store in df variable
df = pd.read_csv(climate_data, names= features)

**Preprocessing**

In [ ]:
# Check for missing:
missing = df.isnull().sum() # There are none
print(missing)


# Check for duplicated:
duplicated = df.duplicated().sum() #There are none
print(duplicated)


#Outlier Detection:
isolation_forest = IsolationForest(contamination="auto",random_state=23)

isolation_forest.fit(df)

predict_df = isolation_forest.predict(df)

df["predict"] = predict_df

outliers = sum(predict_df == -1)

print(outliers)

#Remove Outliers:
df = df[df["predict"]==1] # remove outliers
df = df.drop("predict", axis=1) #remove the predict (1/-1) outlier column

#Feature Selection/Extraction:
variance_level = 5 # this was bassed on print(variance_per_feature)
variance_per_feature = df.var()
print(variance_per_feature)

get_rid = variance_per_feature[variance_per_feature <= variance_level].index.tolist()
print(get_rid)

#Remove Features with lower variance then 5 from the DataFrame
df = df.drop('Precipitation Total mm', axis=1)
df = df.drop('Snowfall Amount cm', axis=1)
df = df.drop('Wind Gust (Min) Km/h', axis=1)
df = df.drop('Wind Speed (Min) Km/h', axis=1)
df = df.drop('Wind Speed (Mean) Km/h', axis=1)


#Feature Engineering:

#Season (Summer/Winter)
temp = 13 # lowest temp in summer (June)
df["Season(Summer (1) or Winter(0))"] = ((df["Temperature (Min) oC"] >= temp).astype(int))

#Temperature Range
df["Temperature Range"] = (df["Temperature (Max) oC."]-df["Temperature (Min) oC"])


#Scaling:

#First determine the distribution to then determine the scaling option
#Shapiro - Wilk Test

normal_distribution_test = stats.shapiro(df)
stat = normal_distribution_test.statistic
p_value = normal_distribution_test.pvalue

print(p_value)   # if p-value > 0.05, fail reject null data is likley normally dis, if p-value <= 0.05 reject null, data is no normally distributed
print(stat)  # indicates how closely the sameple distribuation matches a normal distribution. In general, values closer to 1 suggest that the data is more likey to be normal distributed

#output of the p_value and stat was 5.069605338919966e-115 and 0.5971794215682675, so reject null it is not normally distributed


In [ ]:
#Now apply RobustScaler
updated_features = df.columns.values.tolist()

rbscaler = RobustScaler()

non_binary_features = df.columns[df.columns != "Season(Summer (1) or Winter(0))"]

df[non_binary_features] = rbscaler.fit_transform(df[non_binary_features])

In [ ]:
#Dimension Reduction (PCA)

#Drop the binary column
other_features = []
binary_features = ["Season(Summer (1) or Winter(0))"]

for feature in updated_features:
  if feature not in binary_features:
    other_features.append(feature)

other_features_data = df[other_features]

#PCA
pca = PCA(n_components=0.95) # we want to have 95% of the variance explained
pca_results = pca.fit_transform(other_features_data) # results

components_used = pca.n_components_
print(components_used) # used 7 components to explain 95% of the variance

#Turn results to a dataframe
pca_df = pd.DataFrame(pca_results, columns=["PCA1","PCA2","PCA3","PCA4","PCA5","PCA6","PCA7"])

#Add back in the binary (season)
pca_df["Season"] = df["Season(Summer (1) or Winter(0))"].reset_index(drop=True)


In [ ]:
#Visualize
plt.figure(figsize=(6,4))
sns.scatterplot(x="PCA1",y="PCA2", hue="Season", data=pca_df, s=100, markers="o")
plt.title(label="PCA 1 and 2 Plot")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.show()


plt.figure(figsize=(6,4))
sns.scatterplot(x="PCA1",y="PCA3", hue="Season", data=pca_df, s=100, markers="o")
plt.title(label="PCA 1 and 3 Plot")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 3")
plt.show()


plt.figure(figsize=(6,4))
sns.scatterplot(x="PCA2",y="PCA3", hue="Season", data=pca_df, s=100, markers="o")
plt.title(label="PCA 2 and 3 Plot")
plt.xlabel("PCA Component 2")
plt.ylabel("PCA Component 3")
plt.show()

In [ ]:
#Dimention Reduction (UMAP)

#Finding the optimal number of components
tscores = {}
for x in range(2,11):
    u_map = umap.UMAP(n_components=x)
    umap_results = u_map.fit_transform(df)
    tscores[x] = trustworthiness(df, umap_results)

print(tscores)

#Will use 4 components 0.99191370646281 (trust score), after this the value diminishes with the increase in componetns

#Apply Umap
umap_reduction = umap.UMAP(n_components=4, random_state=23)
umap_reduction_results = umap_reduction.fit_transform(other_features_data)

#Turn Results to DataFrame
umap_df = pd.DataFrame(umap_reduction_results, columns=["Umap1" , "Umap2", "Umap3", "Umap4"])
umap_df["Season"] = df["Season(Summer (1) or Winter(0))"].reset_index(drop=True)



In [ ]:
#Visualize
plt.figure(figsize=(6,4))
sns.scatterplot(x="Umap1",y="Umap2", hue="Season", data=umap_df, s=100, markers="o")
plt.title(label="Umap 1 and 2 Plot")
plt.xlabel("Umap Component 1")
plt.ylabel("Umap Component 2")
plt.show()

plt.figure(figsize=(6,4))
sns.scatterplot(x="Umap1",y="Umap3", hue="Season", data=umap_df, s=100, markers="o")
plt.title(label="Umap 1 and 3 Plot")
plt.xlabel("Umap Component 1")
plt.ylabel("Umap Component 3")
plt.show()

plt.figure(figsize=(6,4))
sns.scatterplot(x="Umap2",y="Umap3", hue="Season", data=umap_df, s=100, markers="o")
plt.title(label="Umap 2 and 3 Plot")
plt.xlabel("Umap Component 2")
plt.ylabel("Umap Component 3")
plt.show()

**Clustering**

In [ ]:
#Clustering - SpectralClustering on (PCA)

#Finding the optimal n_components by finding the highest n_componet and Silhouette Score going through 2 -20 Components
silscores = []

for x in range(2,21):
    spectral_clustering = SpectralClustering(n_components=x, affinity="rbf",random_state =23)
    spectral_clustering_results = spectral_clustering.fit_predict(pca_df)
    score_spc = silhouette_score(pca_df, spectral_clustering_results)
    silscores.append((x, score_spc))

best_pair_spc = max(silscores, key=lambda y: y[1])
print(best_pair_spc)



In [ ]:
#Visualize Clusters - Spectral Clustering
real_spectral_cluster = SpectralClustering(n_components=5, affinity="rbf", random_state=23)
real_spectral_cluster_results = real_spectral_cluster.fit_predict(pca_df)
pca_df["real_spectral_cluster_results"] = real_spectral_cluster_results

plt.figure(figsize=(8,6))
sns.scatterplot(x="PCA1", y="PCA2", hue= "real_spectral_cluster_results", data=pca_df,  s=100, markers="o")
plt.title("Spectral Clustering Results on PCA Comp")
plt.xlabel("PCA1")
plt.ylabel("PCA2")
plt.legend(title="Cluster Labels", loc="upper left", fontsize = "small")
plt.show()

In [ ]:
#Clustering - HDBSCAN on (Umap)

#Finding the optimal parameters for min cluster size and min sample size by maximizing the Silhouette score
mcs = []
ms = []

silscores_2 = []

for x in range (2,21):
    mcs.append(x)
    ms.append(x)

for i in mcs:
    for j in ms:
        h_dbscan = hdbscan.HDBSCAN(min_cluster_size=i,min_samples=j)
        h_dbscan_results = h_dbscan.fit_predict(umap_df)
        if np.any(h_dbscan_results != -1):
            score = silhouette_score(umap_df, h_dbscan_results)
            silscores_2.append((i,j,score))

best_pair_hdbscan = max(silscores_2, key=lambda x: x[2])
print(best_pair_hdbscan)

In [ ]:
#Visualize Clusters - HDBSCAN Clustering
hdbscan_cluster = hdbscan.HDBSCAN(min_cluster_size=15,min_samples=19)
hdbscan_cluster_results = hdbscan_cluster.fit_predict(umap_df)
umap_df["hdbscan_cluster_results"] = hdbscan_cluster_results

plt.figure(figsize=(8,6))
sns.scatterplot(x= "Umap1", y="Umap2", hue="hdbscan_cluster_results", data=umap_df, s=100, markers="o")
plt.title("HDBSCAN clustering on Umap Comp")
plt.xlabel("Umap1")
plt.ylabel("Umap2")
plt.legend(title="Cluster Labels")
plt.show()


In [ ]:
#Clustering - Affinity Propagation on (PCA)

#drop the added cluster column form spectral clustering
pca_df.drop("real_spectral_cluster_results", axis=1)

#Applying Affinity
clustering_pca = AffinityPropagation(random_state=23)
clustering_pca_results = clustering_pca.fit_predict(pca_df)
pca_df["Clusters Affinity"] = clustering_pca_results

In [ ]:
#Clustering - Affinity Propagation on (Umap)
#drop the added cluster column from HDBSCAN
umap_df.drop("hdbscan_cluster_results", axis=1)

#Applying Affinity
clustering_umap = AffinityPropagation(random_state=23)
clustering_umap_results = clustering_umap.fit_predict(umap_df)
umap_df["Clusters Affinity"] = clustering_umap_results

In [ ]:
#Visualize

#Affinity Propagation on (PCA)
plt.figure(figsize=(8,6))
sns.scatterplot(x="PCA1", y="PCA2", hue ="Clusters Affinity", data=pca_df, s=100, markers="o")
plt.title("Affinity Propagation Clustering on PCA Comp")
plt.legend(title="Cluster Labels")
plt.xlabel("PCA1")
plt.ylabel("PCA2")
plt.show()


plt.figure(figsize=(8,6))
sns.scatterplot(x="Umap1", y="Umap2", hue ="Clusters Affinity", data=umap_df, s=100, markers="o")
plt.title("Affinity Propagation Clustering on Umap Comp")
plt.legend(title="Cluster Labels")
plt.xlabel("Umap1")
plt.ylabel("Umap2")
plt.show()

In [ ]:
#Clusters Evaluation - Davies-Bouldin


#Spectral Clustering
db_spectral_clustering = davies_bouldin_score(pca_results,real_spectral_cluster_results)

#HDBSCAN Clustering
db_hdbscan_clustering = davies_bouldin_score(umap_reduction_results, hdbscan_cluster_results)

#Affinity Propagation Clustering on (PCA)
db_ap_pca = davies_bouldin_score(pca_results,clustering_pca_results)

#Affinity Propagation Clustering on (Umap)
db_ap_umap = davies_bouldin_score(umap_reduction_results,clustering_umap_results)


#Print

print(f"The Davies Bouldin Scores: Spectral Clustering: {db_spectral_clustering}, HDBSCAN Clustering: {db_hdbscan_clustering}, Affinity Propagation Clustering on (PCA): {db_ap_pca},Affinity Propagation Clustering on (Umap):{db_ap_umap}")